<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/FESC_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
time.sleep(5)

!unzip /content/FESC_CaFE.zip

In [7]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/FESC/FESC_splits/ANRO/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/FESC/FESC_splits/ANRO/test.json",'r') as f:
    test_data = json.load(f)

In [8]:
train_data

{'TIRE_3872_1': {'file_path': 'FESC/16kh_resampled/TIRE_3872_1.WAV',
  'length': 3.4863945578231292,
  'label': '1'},
 'TUVA_4089_3': {'file_path': 'FESC/16kh_resampled/TUVA_4089_3.WAV',
  'length': 3.5664399092970522,
  'label': '3'},
 'TIRE_3623_5': {'file_path': 'FESC/16kh_resampled/TIRE_3623_5.WAV',
  'length': 3.7662585034013607,
  'label': '5'},
 'TIPE_3131_3': {'file_path': 'FESC/16kh_resampled/TIPE_3131_3.WAV',
  'length': 3.3277551020408165,
  'label': '3'},
 'TIRE_3615_1': {'file_path': 'FESC/16kh_resampled/TIRE_3615_1.WAV',
  'length': 8.525532879818595,
  'label': '1'},
 'TUVA_4009_1': {'file_path': 'FESC/16kh_resampled/TUVA_4009_1.WAV',
  'length': 3.4434467120181407,
  'label': '1'},
 'RIGR_2895_4': {'file_path': 'FESC/16kh_resampled/RIGR_2895_4.WAV',
  'length': 3.3902947845804987,
  'label': '4'},
 'JARA_1035_5': {'file_path': 'FESC/16kh_resampled/JARA_1035_5.WAV',
  'length': 2.7300680272108844,
  'label': '5'},
 'MIKO_1611_3': {'file_path': 'FESC/16kh_resampled/MIKO_1

In [9]:
test_data

{'ANRO_256_1': {'file_path': 'FESC/16kh_resampled/ANRO_256_1.WAV',
  'length': 2.125079365079365,
  'label': '1'},
 'ANRO_313_4': {'file_path': 'FESC/16kh_resampled/ANRO_313_4.WAV',
  'length': 1.9469387755102041,
  'label': '4'},
 'ANRO_273_3': {'file_path': 'FESC/16kh_resampled/ANRO_273_3.WAV',
  'length': 2.0040362811791383,
  'label': '3'},
 'ANRO_418_5': {'file_path': 'FESC/16kh_resampled/ANRO_418_5.WAV',
  'length': 2.200544217687075,
  'label': '5'},
 'ANRO_286_1': {'file_path': 'FESC/16kh_resampled/ANRO_286_1.WAV',
  'length': 1.9787755102040816,
  'label': '1'},
 'ANRO_362_5': {'file_path': 'FESC/16kh_resampled/ANRO_362_5.WAV',
  'length': 2.838140589569161,
  'label': '5'},
 'ANRO_78_2': {'file_path': 'FESC/16kh_resampled/ANRO_78_2.WAV',
  'length': 3.394467120181406,
  'label': '2'},
 'ANRO_308_1': {'file_path': 'FESC/16kh_resampled/ANRO_308_1.WAV',
  'length': 6.325260770975056,
  'label': '1'},
 'ANRO_306_1': {'file_path': 'FESC/16kh_resampled/ANRO_306_1.WAV',
  'length': 

In [10]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [11]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [12]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 3229
    })
    test: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 450
    })
    dev: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 450
    })
})

In [13]:
def add_base_path(example):
    example['file_path'] = '/content/FESC/FESC_splits/ANRO/' + example['file_path']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/3229 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

{'id': 'TIRE_3872_1',
 'file_path': '/content/FESC/FESC_splits/ANRO/FESC/16kh_resampled/TIRE_3872_1.WAV',
 'length': 3.4863945578231292,
 'label': '1'}

In [18]:
import librosa
import os

def load_audio(example):
  # Check if the file exists
  if os.path.exists(example['file_path']):
    audio_array, sample_rate = librosa.load(example['file_path'], sr=16000)
    example["audio_array"] = audio_array
    example["sample_rate"] = sample_rate
    return example
  else:
    print(f"Warning: File not found: {example['file_path']}")
    return example  # Or handle the missing file in a different way

In [16]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['file_path'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [19]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/3229 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 3229
    })
    test: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 450
    })
    dev: Dataset({
        features: ['id', 'file_path', 'length', 'label'],
        num_rows: 450
    })
})

In [20]:
dataset['train'][0].keys()

dict_keys(['id', 'file_path', 'length', 'label'])

In [21]:
import torch
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, TrainingArguments, Trainer
!pip install datasets --upgrade

In [23]:
def is_common(emotion):
    if emotion == "neutral" or emotion == "sadness" or emotion == "happiness" or emotion == "anger":
        return 1
    else:
        return 0


In [22]:
def update_fesc_label(label, column="emo"):
    if label == "1":
        return "anger"
    elif label == "2":
        return "happiness"
    elif label == "3":
        return "neutral"
    elif label == "4":
        return "sadness"

In [ ]:
def fesc(session):
    SESSION_FOLDER_MAP = {
        1: "TIRE", 2: "TIPE", 3: "JARA", 4: "MIKO", 5: "ANRO",
        6: "RIGR", 7: "TUVA", 8: "PEKO", 9: "JAKA"
    }
    FOLDER_NAME = SESSION_FOLDER_MAP.get(session, "ERROR")

    train_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/Finnish-emotion-spilits/{FOLDER_NAME}/train.json"
    test_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/Finnish-emotion-spilits/{FOLDER_NAME}/test.json"
    dev_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/Finnish-emotion-spilits/{FOLDER_NAME}/dev.json"

    with open(train_json_file, "r") as f:
        train_data = json.load(f)
    with open(test_json_file, "r") as f:
        test_data = json.load(f)
    with open(dev_json_file, "r") as f:
        dev_data = json.load(f)


    train_df = pd.DataFrame.from_dict(train_data, orient="index").reset_index()
    test_df = pd.DataFrame.from_dict(test_data, orient="index").reset_index()
    dev_df = pd.DataFrame.from_dict(dev_data, orient="index").reset_index()

    train_df["audio"] = train_df["file_path"].apply(update_file_path)
    test_df["audio"] = test_df["file_path"].apply(update_file_path)
    dev_df["audio"] = dev_df["file_path"].apply(update_file_path)

    train_df = train_df.rename(columns={'index': 'file_id', 'label': 'emo'})
    test_df = test_df.rename(columns={'index': 'file_id', 'label': 'emo'})
    dev_df = dev_df.rename(columns={'index': 'file_id', 'label': 'emo'})

    train_df = train_df.loc[train_df['emo'] != '5'].reset_index(drop=True)
    test_df = test_df.loc[test_df['emo'] != '5'].reset_index(drop=True)
    dev_df = dev_df.loc[dev_df['emo'] != '5'].reset_index(drop=True)

    train_df["emo"] = train_df["emo"].apply(update_fesc_label)
    test_df["emo"] = test_df["emo"].apply(update_fesc_label)
    dev_df["emo"] = dev_df["emo"].apply(update_fesc_label)

    labels = ['anger', 'happiness', 'neutral', 'sadness'] # ["1", "2", "3", "4"]
    label2id, id2label = dict(), dict()

    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label

    train_df["label"] = train_df["emo"].apply(to_label, args=(label2id,))
    test_df["label"] = test_df["emo"].apply(to_label, args=(label2id,))
    dev_df["label"] = dev_df["emo"].apply(to_label, args=(label2id,))

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']])
    train_audio_data = train_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']])
    test_audio_data = test_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    dev_audio_data = Dataset.from_pandas(dev_df[['audio', 'label']])
    dev_audio_data = dev_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    ds = DatasetDict({
        'train' : train_audio_data,
        'test' : test_audio_data,
        'dev' : dev_audio_data
    })

    return label2id, id2label, ds

In [24]:
import evaluate
accuracy = evaluate.load("accuracy")

In [25]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer, AutoFeatureExtractor

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

In [ ]:
training_args = TrainingArguments(
    output_dir="FESC_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub()


In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import recall_score


In [ ]:
def collate_fn(batch):
batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
inputs = [torch.tensor(example["input_values"]) for example in batch]
labels = [int(example["label"]) for example in batch]
inputs = pad_sequence(inputs, batch_first=True)
return {"input_values": inputs, "label": torch.tensor(labels)}

In [ ]:
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()

    total_predictions = 0
    correct_predictions = 0
    total_train_loss = 0

    # train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_OF_EPOCHS}, Training", leave=False)

    all_actual_labels = []
    all_predictions = []
    train_confidence_scores = []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        outputs = outputs.logits

        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        train_loader.set_postfix(loss=loss.item())

        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        total_train_loss += loss.item()

        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

        max_values, max_indices = torch.softmax(outputs, dim=1).max(dim=1)
        probability_label_tuples = [(max_value.item(), max_index.item()) for max_value, max_index in zip(max_values, max_indices)]
        train_confidence_scores.extend(probability_label_tuples)

    total_train_loss /= len(train_loader)
    train_accuracy = correct_predictions / total_predictions

    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_actual_labels, all_predictions, average='weighted')

    return (unweighted_recall, weighted_recall, train_accuracy, total_train_loss, train_confidence_scores)